In [1]:
def test_model(ablation=1000, B=32, epochs=3, filters=64, kernel_size=3, dense_neurons=2048, 
              conv1d_set1=1, conv1d_set2=1, maxpool_1=False):
    from hdflogv1 import HDFSLogv1
    import tensorflow as tf
    import numpy as np
    tf.random.set_seed(123)
    import pickle
    with open('../data/hdfs_log_obj.pkl', 'rb') as f:
        hdfslogs = pickle.load(f)
    ablation_data = hdfslogs.get_train_test_data(ablation=ablation)
    x_train, y_train, x_test, y_test = hdfslogs.get_padded_train_test_data(ablation=ablation)
    train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_data = train_data.shuffle(buffer_size=y_train.shape[0]).batch(B, drop_remainder=True)
    print(train_data)
    
    test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_data = test_data.shuffle(buffer_size=y_test.shape[0]).batch(B, drop_remainder=True)
    print(test_data)
    
    
    tk = hdfslogs.tk    
    vocab_size = len(tk.word_index)
    print(f'vocab_size: {vocab_size}')
    char_onehot = vocab_size
    
    embedding_weights = []
    embedding_weights.append(np.zeros(vocab_size))
    for char, i in tk.word_index.items(): # from 1 to 51
        onehot = np.zeros(vocab_size)
        onehot[i-1] = 1
        embedding_weights.append(onehot)
    embedding_weights = np.array(embedding_weights)
    
    input_size = [x_train.shape[1], x_train.shape[2]]
    embedding_size = vocab_size

    embedding_layer = tf.keras.layers.Embedding(vocab_size+1,
                                                embedding_size,
                                                input_length=input_size,
                                                weights = [embedding_weights])
    
    
    
    
    inputs = tf.keras.layers.Input(batch_shape=(B, x_train.shape[1], x_train.shape[2]), dtype='float64' )
    x = tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                    output_dim=embedding_size,
                                    input_length=x_train.shape[2],
                                    weights = [embedding_weights],
                                    )(inputs)
    for _ in range(conv1d_set1):
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    if maxpool_1:
        x = tf.keras.layers.MaxPooling2D(pool_size=(1, x_train.shape[2]))(x)
        x = tf.reshape(x, (B, x_train.shape[1], filters))        
        for _ in range(conv1d_set2):
            x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
        x = tf.keras.layers.MaxPooling1D(pool_size=(x_train.shape[1]) )(x)    
    if not maxpool_1:
        x = tf.keras.layers.Flatten()(x)       
    x = tf.keras.layers.Dense(dense_neurons)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    print(model.summary())
    model.compile(optimizer='adam', 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    hist = model.fit(train_data, validation_data=test_data, epochs=epochs) 
    return model, hist

In [ ]:
test_model(ablation=4000, B=250, kernel_size=3, epochs=16, dense_neurons=2048, conv1d_set1=3,conv1d_set2=3, maxpool_1=True )

getting ablation data: 4000
4000 12838
train_test_data done: 0.019000768661499023
RAM usage train_test_data:  72
length of train  sequence original 4
length of train  sequence original 25
length of train  sequence original 33
length of train  sequence original 21
length of train  sequence original 2
length of train sequence padded 32
length of train sequence padded 32
length of train sequence padded 32
length of train sequence padded 32
length of train sequence padded 32
len of test seq after padding 32
len of test seq after padding 32
len of test seq after padding 32
len of test seq after padding 32
len of test seq after padding 32
padded_train_test_data done: 6.683482885360718
RAM usage padded_train_test_data:  72
<BatchDataset shapes: ((250, 32, 64), (250,)), types: (tf.int32, tf.int64)>
<BatchDataset shapes: ((250, 32, 64), (250,)), types: (tf.int32, tf.int64)>
vocab_size: 42
Model: "functional_1"
_________________________________________________________________
Layer (type)       